<a href="https://colab.research.google.com/github/Marchinski/AI_Models/blob/main/VGG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
import os
import pickle
import time, datetime

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm
import tensorflow as tf

### Methods

In [16]:
def show_train_hist(hist, show=False, save=False, path='Train_hist.png'):
    """Loss tracker
    
    Plot the losses of the networks independently to see the trend
    
    Arguments:
        hist {[dict]} -- Tracking variables
    
    Keyword Arguments:
        show {bool} -- If to display the figure (default: {False})
        save {bool} -- If to store the figure (default: {False})
        path {str} -- path to store the figure (default: {'Train_hist.png'})
    """
    x = range(len(hist['VGG_losses']))

    y1 = hist['VGG_losses']
    y2 = hist['ResNet_losses']
    y3 = hist['GGL_Net_losses']

    plt.plot(x, y1, label='VGG_loss')
    plt.plot(x, y2, label='ResNet_loss')
    plt.plot(x, y3, label='GoogLeNet_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

### Data Loading and processing

In [3]:
# initialise the device for training, if gpu is available, device = 'cuda', else: device = 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

data_dir = './MNIST_data/'
save_dir = './Results/'
model_dir = './Models/'
# create folder if not exist
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# training parameters
batch_size = 256
learning_rate = 0.01
epochs = 100

# construct the datasets and data loaders
transform = transforms.Compose([transforms.Resize(64),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,), std=(0.5,))])
tv_data = datasets.MNIST(root=data_dir, train=True, transform=transform, download=True)
train_data, validation_data = torch.utils.data.random_split(tv_data, [50000, 10000])

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=batch_size, shuffle=True)

test_data = datasets.MNIST(root=data_dir, train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

cuda


# Model loading and training

In [4]:
# tracking variables
train_hist = {}
train_hist['VGG_losses'] = []
train_hist['ResNet_losses'] = []
train_hist['GGL_Net_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

# Loss function
criterion = nn.CrossEntropyLoss().to(device)

### VGG Model methods

In [5]:
def LoadVGG(device):
    VGG_net = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=False)
    VGG_net = VGG_net.to(device)

    VGG_optimizer = optim.SGD(VGG_net.parameters(), lr=0.01, momentum=0.9)
    VGG_scheduler = optim.lr_scheduler.ReduceLROnPlateau(VGG_optimizer, mode='min', patience=5, verbose=True)

    return VGG_net, VGG_optimizer, VGG_scheduler

def TrainVGG(VGG_net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader):
    print('\nTraining VGG')
    train_hist['per_epoch_ptimes'].clear()
    start_time = time.time()
    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Training starting ',str(dt_string))
    for epoch in range(epochs):
        Train_Loss = []
        Val_Loss = []
        epoch_start_time = time.time()
        now = datetime.datetime.now()
        print('Epoch %d starting at: %02d:%02d:%02d'%(epoch+1, now.hour, now.minute, now.second))
        VGG_net.train()
        # Training
        for (train_images, train_labels) in tqdm(train_loader):
            train_images = train_images.repeat(1, 3, 1, 1)
            train_images = train_images.to(device)
            train_labels = train_labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_outputs = VGG_net(train_images)
            train_loss = criterion(train_outputs, train_labels)
            train_loss.backward()
            optimizer.step()

            Train_Loss.append(train_loss.item())
        
        VGG_net.eval()
        # Validation
        for (val_images, val_labels) in tqdm(validation_loader):
            val_images = val_images.repeat(1, 3, 1, 1)
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            # calculate outputs by running images through the network
            val_outputs = VGG_net(val_images)
            val_loss = criterion(val_outputs, val_labels)
            
            Val_Loss.append(val_loss.item())

        # Learning Rate Adaptation step
        # changes LR on flat plateau
        scheduler.step(np.mean(Val_Loss))

        epoch_loss = np.mean(Train_Loss)  # mean generator loss for the epoch
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print("\nEpoch %d of %d, %.2f s, with average training loss: %.8f, and validation loss: %.8f" 
              % (epoch + 1, epochs, per_epoch_ptime, epoch_loss, np.mean(Val_Loss)))

        # record the loss for every epoch
        train_hist['VGG_losses'].append(epoch_loss)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Finished Training ',str(dt_string))

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))

### ResNet Model methods

In [10]:
def LoadResNet(device):
    ResNet_net = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=False)
    ResNet_net = ResNet_net.to(device)

    ResNet_optimizer = optim.SGD(ResNet_net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
    ResNet_scheduler = optim.lr_scheduler.ReduceLROnPlateau(ResNet_optimizer, mode='min', patience=5, verbose=True)

    return ResNet_net, ResNet_optimizer, ResNet_scheduler

def TrainResNet(ResNet_net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader):
    print('\nTraining ResNet')
    train_hist['per_epoch_ptimes'].clear()
    start_time = time.time()
    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Training starting ',dt_string)
    for epoch in range(epochs):
        Train_Loss = []
        Val_Loss = []
        epoch_start_time = time.time()
        now = datetime.datetime.now()
        print('Epoch %d starting at: %02d:%02d:%02d'%(epoch+1, now.hour, now.minute, now.second))
        ResNet_net.train()
        # Training
        for (train_images, train_labels) in tqdm(train_loader):
            train_images = train_images.repeat(1, 3, 1, 1)
            train_images = train_images.to(device)
            train_labels = train_labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_outputs = ResNet_net(train_images)
            train_loss = criterion(train_outputs, train_labels)
            train_loss.backward()
            optimizer.step()

            Train_Loss.append(train_loss.item())
        
        ResNet_net.eval()
        # Validation
        for (val_images, val_labels) in tqdm(validation_loader):
            val_images = val_images.repeat(1, 3, 1, 1)
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            # calculate outputs by running images through the network
            val_outputs = ResNet_net(val_images)
            val_loss = criterion(val_outputs, val_labels)
            
            Val_Loss.append(val_loss.item())

        # Learning Rate Adaptation step
        # changes LR on flat plateau
        scheduler.step(np.mean(Val_Loss))

        epoch_loss = np.mean(Train_Loss)  # mean generator loss for the epoch
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print("\nEpoch %d of %d, %.2f s, with average training loss: %.8f, and validation loss: %.8f" 
              % (epoch + 1, epochs, per_epoch_ptime, epoch_loss, np.mean(Val_Loss)))

        # record the loss for every epoch
        train_hist['ResNet_losses'].append(epoch_loss)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Finished Training ',str(dt_string))

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))

### GoogLeNet Model Methods

In [12]:
def LoadGLNet(device):
    GLNet_net = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=False)
    GLNet_net = GLNet_net.to(device)

    GLNet_optimizer = optim.SGD(GLNet_net.parameters(), lr=0.01, momentum=0.9)
    GLNet_scheduler = optim.lr_scheduler.StepLR(GLNet_optimizer, step_size=8, gamma=0.96, verbose=True)

    return GLNet_net, GLNet_optimizer, GLNet_scheduler

def TrainGLNet(GLNet_net, optimizer, scheduler, criterion, epochs, train_loader):
    print('\nTraining GoogLeNet')
    train_hist['per_epoch_ptimes'].clear()
    start_time = time.time()
    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Training starting ',dt_string)
    for epoch in range(epochs):
        GLNet_net.train()
        Train_Loss = []
        Val_Loss = []
        epoch_start_time = time.time()
        now = datetime.datetime.now()
        print('Epoch %d starting at: %02d:%02d:%02d'%(epoch+1, now.hour, now.minute, now.second))
        # Training
        for (train_images, train_labels) in tqdm(train_loader):
            train_images = train_images.repeat(1, 3, 1, 1)
            train_images = train_images.to(device)
            train_labels = train_labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            train_outputs = GLNet_net(train_images)
            train_outputs = train_outputs.logits
            train_loss = criterion(train_outputs, train_labels)
            train_loss.backward()
            optimizer.step()

            Train_Loss.append(train_loss.item())

        # Learning Rate Adaptation step
        # changes LR every 8 epochs
        scheduler.step()

        epoch_loss = np.mean(Train_Loss)  # mean generator loss for the epoch
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print("\nEpoch %d of %d, %.2f s, with average training loss: %.8f" 
              % (epoch + 1, epochs, per_epoch_ptime, epoch_loss))

        # record the loss for every epoch
        train_hist['GGL_Net_losses'].append(epoch_loss)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print('Finished Training ',str(dt_string))

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))

# Models training

### VGG Training

In [8]:
# Load and trian VGG
net, optimizer, scheduler = LoadVGG(device)
TrainVGG(net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader)
torch.save(net.state_dict(), model_dir + '/VGG_model.pth')
del net
del optimizer
del scheduler

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
  0%|          | 0/196 [00:00<?, ?it/s]


Training VGG
Training starting  09/05/2021 19:07:28
Epoch 1 starting at: 19:07:28


  0%|          | 0/196 [00:00<?, ?it/s]


Epoch 1 of 100, 67.83 s, with average training loss: 3.76883705, and validation loss: 2.32187066
Epoch 2 starting at: 19:08:36


  0%|          | 0/196 [00:00<?, ?it/s]

...


Epoch 98 of 100, 67.90 s, with average training loss: 0.00198397, and validation loss: 0.07438689
Epoch 99 starting at: 20:58:10


  0%|          | 0/196 [00:00<?, ?it/s]


Epoch 99 of 100, 67.79 s, with average training loss: 0.00206194, and validation loss: 0.07541705
Epoch 100 starting at: 20:59:18


100%|██████████| 40/40 [00:05<00:00,  7.47it/s]



Epoch 100 of 100, 67.83 s, with average training loss: 0.00214176, and validation loss: 0.07440045
Finished Training  09/05/2021 21:00:26
Avg per epoch ptime: 67.78, total 100 epochs ptime: 6778.11


### ResNet Training

In [13]:
# Load and train ResNet
net, optimizer, scheduler = LoadResNet(device)
TrainResNet(net, optimizer, scheduler, criterion, epochs, train_loader, validation_loader)
torch.save(net.state_dict(), model_dir + '/ResNet_model.pth')
del net
del optimizer
del scheduler

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
  1%|          | 1/196 [00:00<00:32,  6.09it/s]


Training ResNet
Training starting  09/05/2021 21:28:42
Epoch 1 starting at: 21:28:42


  1%|          | 1/196 [00:00<00:29,  6.52it/s]


Epoch 1 of 100, 32.65 s, with average training loss: 0.31101736, and validation loss: 0.05770844
Epoch 2 starting at: 21:29:15


  1%|          | 1/196 [00:00<00:29,  6.53it/s]

...


Epoch 98 of 100, 32.15 s, with average training loss: 0.00012463, and validation loss: 0.04218569
Epoch 99 starting at: 22:21:24


  1%|          | 1/196 [00:00<00:29,  6.56it/s]


Epoch 99 of 100, 32.16 s, with average training loss: 0.00012320, and validation loss: 0.04141310
Epoch 100 starting at: 22:21:56


100%|██████████| 40/40 [00:02<00:00, 13.35it/s]



Epoch 100 of 100, 32.19 s, with average training loss: 0.00012513, and validation loss: 0.03945253
Finished Training  09/05/2021 22:22:28
Avg per epoch ptime: 32.26, total 100 epochs ptime: 3226.18


### GoogLeNet Training

In [14]:
# Load and train GoogLeNet
net, optimizer, scheduler = LoadGLNet(device)
TrainGLNet(net, optimizer, scheduler, criterion, epochs, train_loader)
torch.save(net.state_dict(), model_dir + '/GoogLeNet_model.pth')
del net
del optimizer
del scheduler

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:79: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
  1%|          | 1/196 [00:00<00:28,  6.88it/s]

Adjusting learning rate of group 0 to 1.0000e-02.

Training GoogLeNet
Training starting  09/05/2021 22:36:15
Epoch 1 starting at: 22:36:15


  1%|          | 1/196 [00:00<00:25,  7.51it/s]

Adjusting learning rate of group 0 to 1.0000e-02.

Epoch 1 of 100, 25.84 s, with average training loss: 0.40671538
Epoch 2 starting at: 22:36:41


  1%|          | 1/196 [00:00<00:25,  7.58it/s]

...

Adjusting learning rate of group 0 to 6.1271e-03.

Epoch 98 of 100, 26.03 s, with average training loss: 0.00001807
Epoch 99 starting at: 23:18:28


  1%|          | 1/196 [00:00<00:25,  7.53it/s]

Adjusting learning rate of group 0 to 6.1271e-03.

Epoch 99 of 100, 25.78 s, with average training loss: 0.00001808
Epoch 100 starting at: 23:18:53


100%|██████████| 196/196 [00:25<00:00,  7.57it/s]


Adjusting learning rate of group 0 to 6.1271e-03.

Epoch 100 of 100, 25.90 s, with average training loss: 0.00001667
Finished Training  09/05/2021 23:19:19
Avg per epoch ptime: 25.84, total 100 epochs ptime: 2584.28


In [18]:
# Show training results
show_train_hist(train_hist, save=True, path=save_dir + '/MNIST_train_hist.png')

### Model testing